In [35]:
import librosa
import pandas as pd
import numpy as np
import os
import pathlib
from pathlib import Path
import pickle
from tinytag import TinyTag
from pydub import AudioSegment
import ntpath
from pydub.utils import make_chunks
from numpy  import array

In [36]:
from itertools import tee, islice, chain
from math import atan2,degrees

In [42]:
pathlist = Path("D:/learning/DataScience/data/musicFiles").glob('**/*.mp3')


In [41]:
##15
header = 'filename artist duration albumartist bitrate comment filesize genre title track samplerate track_total year audio_waveform sampling_rate vocals '
## 26
fe_col_header = 'audio_waveform_split chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate mfcc rmse chroma_cqt chroma_cens melspectrogram spectral_contrast spectral_flatness poly tonal_centroid tempogram onset_detect onset_strength onset_strength_multi beat tempo harmonic_elements percussive_elements delta '

fe_col_header_angle = 'audio_waveform_split_angle chroma_stft_angle spectral_centroid_angle spectral_bandwidth_angle rolloff_angle zero_crossing_rate_angle mfcc_angle rmse_angle chroma_cqt_angle chroma_cens_angle melspectrogram_angle spectral_contrast_angle spectral_flatness_angle poly_angle tonal_centroid_angle tempogram_angle onset_detect_angle onset_strength_angle onset_strength_multi_angle beat_angle tempo_angle harmonic_elements_angle percussive_elements_angle delta_angle'

header += fe_col_header
header += fe_col_header_angle


#header += ' audio_waveform_split chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate mfcc rmse chroma_cqt chroma_cens melspectrogram spectral_contrast spectral_flatness poly tonal_centroid tempogram onset_detect onset_strength onset_strength_multi beat tempo harmonic_elements percussive_elements delta vocals'
header = header.split()
print(len(header))
fe_col_header = fe_col_header.split()

df = pd.DataFrame(columns=header)
df.columns

64


Index(['filename', 'artist', 'duration', 'albumartist', 'bitrate', 'comment',
       'filesize', 'genre', 'title', 'track', 'samplerate', 'track_total',
       'year', 'audio_waveform', 'sampling_rate', 'vocals',
       'audio_waveform_split', 'chroma_stft', 'spectral_centroid',
       'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc', 'rmse',
       'chroma_cqt', 'chroma_cens', 'melspectrogram', 'spectral_contrast',
       'spectral_flatness', 'poly', 'tonal_centroid', 'tempogram',
       'onset_detect', 'onset_strength', 'onset_strength_multi', 'beat',
       'tempo', 'harmonic_elements', 'percussive_elements', 'delta',
       'audio_waveform_split_angle', 'chroma_stft_angle',
       'spectral_centroid_angle', 'spectral_bandwidth_angle', 'rolloff_angle',
       'zero_crossing_rate_angle', 'mfcc_angle', 'rmse_angle',
       'chroma_cqt_angle', 'chroma_cens_angle', 'melspectrogram_angle',
       'spectral_contrast_angle', 'spectral_flatness_angle', 'poly_angle',
       'ton

In [39]:
# Determines the angle of a straight line drawn between point one and two.
def getAngleBetweenTwoPoints(p1x,p1y,p2x,p2y):
    xDiff = p2x - p1x
    yDiff = p2y - p1y
    angle = degrees(atan2(yDiff, xDiff))
    if(angle >= 0):
        return angle
    else:
        return (angle+360)
def getPreviousAndNext(iterable_data):
    prevs, items, nexts = tee(iterable_data, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)
def getArrayOfAnglesBetweenConsecutivepointsInArray(iterable_data):
    xx = 1
    yy = 2
    np_arr =  np.array([])
    for previous, item, nxt in getPreviousAndNext(iterable_data):
        if(nxt == None):
            np_arr = np.append(np_arr,0)
        else:
            tempvar = getAngleBetweenTwoPoints(xx,item,yy,nxt)
            np_arr = np.append(np_arr,tempvar)
    return np_arr

In [43]:
sr_no = 0
for path in pathlist:

    sr_no = sr_no + 1
    path_in_str = str(path)
#     print(path)
    songname = path_in_str.replace(os.sep, '/')
    tag = TinyTag.get(songname)
    ###-----List to store all the metadata and features for a audio file------
    featurelist = []
    vocals= []
    featurelist.append(ntpath.basename((songname)))
    ###-----MP3 file cobersion to wave format for better analysis-------------
    sound = AudioSegment.from_mp3(songname)

    sound.export("final.wav", format="wav")
    y, sr = librosa.load("final.wav", mono=True)
    stft = librosa.feature.chroma_stft(y=y, sr=sr)
    S_full, phase = librosa.magphase(stft)
      # We'll compare frames using cosine similarity, and aggregate similar frames
    # by taking their (per-frequency) median value.
    #
    # To avoid being biased by local continuity, we constrain similar frames to be
    # separated by at least 2 seconds.
    #
    # This suppresses sparse/non-repetetitive deviations from the average spectrum,
    # and works well to discard vocal elements.

    S_filter = librosa.decompose.nn_filter(S_full,aggregate=np.median,metric='cosine',width=int(librosa.time_to_frames(2, sr=sr)))

    # The output of the filter shouldn't be greater than the input
    # if we assume signals are additive.  Taking the pointwise minimium
    # with the input spectrum forces this.
    S_filter = np.minimum(S_full, S_filter)
    # We can also use a margin to reduce bleed between the vocals and instrumentation masks.
    # Note: the margins need not be equal for foreground and background separation
    margin_i, margin_v = 2, 10
    power = 2

    # mask_i = librosa.util.softmask(S_filter,
    #                                margin_i * (S_full - S_filter),
    #                                power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,margin_v * S_filter,power=power)

    # Once we have the masks, simply multiply them with the input spectrum
    # to separate the components

    S_foreground = mask_v * S_full
    vocals.append(np.mean(S_foreground))

    ###-----------------------Audio Metadata-----12-----------------------------
    featurelist.append(tag.artist)
    featurelist.append(tag.duration)
    featurelist.append(tag.albumartist)
    featurelist.append(tag.bitrate)
    featurelist.append(tag.comment)
    featurelist.append(tag.filesize)
    featurelist.append(tag.genre)
    featurelist.append(tag.title) 
    featurelist.append(tag.track) 
    featurelist.append(tag.samplerate) 
    featurelist.append(tag.track_total) 
    featurelist.append(tag.year)
    ###-----------------------Audio Features----------------------2------------
    featurelist.append(pickle.dumps(y,protocol = -1) )
    featurelist.append(sr)
    featurelist.append(pickle.dumps(vocals,protocol = -1))
    

    
    audio_waveform_split = []
    chroma_stft = []
    spectral_centroid = []
    spectral_bandwidth = []
    rolloff= [] 
    zero_crossing_rate= [] 
    mfcc= [] 
    rmse= [] 
    chroma_cqt= [] 
    chroma_cens= [] 
    melspectrogram= [] 
    spectral_contrast= [] 
    spectral_flatness= [] 
    poly= [] 
    tonal_centroid= [] 
    tempogram= [] 
    onset_detect= [] 
    onset_strength= [] 
    onset_strength_multi= [] 
    beat= [] 
    tempo= [] 
    harmonic_elements= [] 
    percussive_elements= [] 
    delta= [] 

    
    
    #--------------------------------------------------------------------------------------------------------
    audio_waveform_split_angle = []
    chroma_stft_angle = []
    spectral_centroid_angle = []
    spectral_bandwidth_angle = []
    rolloff_angle= [] 
    zero_crossing_rate_angle= [] 
    mfcc_angle= [] 
    rmse_angle= [] 
    chroma_cqt_angle= [] 
    chroma_cens_angle= [] 
    melspectrogram_angle= [] 
    spectral_contrast_angle= [] 
    spectral_flatness_angle= [] 
    poly_angle= [] 
    tonal_centroid_angle= [] 
    tempogram_angle= [] 
    onset_detect_angle= [] 
    onset_strength_angle= [] 
    onset_strength_multi_angle= [] 
    beat_angle= [] 
    tempo_angle= [] 
    harmonic_elements_angle= [] 
    percussive_elements_angle= [] 
    delta_angle= [] 
#     vocals_angle= []
    chunk_length_ms = 5000 # pydub calculates in millisec
    chunks = make_chunks(sound, chunk_length_ms) #Make chunks of five sec

    #Export all of the individual chunks as wav files

    for i, chunk in enumerate(chunks):
        print(i)
        chunk.export("chunk.wav", format="wav")
        y, sr = librosa.load("chunk.wav", mono=True)
        audio_waveform_split.append(np.mean(y))
        stft = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_stft.append(np.mean(stft))
        
        spectral_centroid.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)))
        spectral_bandwidth.append(np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)))
        rolloff.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)))
        
        zero_crossing_rate.append(np.mean(librosa.feature.zero_crossing_rate(y)))
        mfcc.append(np.mean(librosa.feature.mfcc(y=y, sr=sr)))
        rmse.append(np.mean(librosa.feature.rmse(y)))
        chroma_cqt.append(np.mean(librosa.feature.chroma_cqt(y , sr=sr))) 
        chroma_cens.append(np.mean(librosa.feature.chroma_cens(y , sr=sr)))
        melspectrogram.append(np.mean(librosa.feature.melspectrogram(y, sr=sr)))
        spectral_contrast.append(np.mean(librosa.feature.spectral_contrast(y , sr=sr)))
        spectral_flatness.append(np.mean(librosa.feature.spectral_flatness(y)))
        poly.append(np.mean(librosa.feature.poly_features(y)))
        tonal_centroid.append(np.mean(librosa.feature.tonnetz(y)))
        tempogram1 = librosa.feature.tempogram(y)
        tempogram.append(np.mean(tempogram1))
        onset_detect.append(np.mean(librosa.onset.onset_detect(y)))
        onset_strength.append(np.mean(librosa.onset.onset_strength(y)))
        onset_strength_multi.append(np.mean(librosa.onset.onset_strength_multi(y)))
        
        a1,b1 = librosa.beat.beat_track(y)
        beat.append(np.mean(b1))
        tempo.append(np.mean(librosa.beat.tempo(y)))
        harmonic_elements.append(np.mean(librosa.effects.hpss(y)))
        percussive_elements.append(np.mean(librosa.effects.percussive(y)))
#         delta.append(np.mean(librosa.feature.delta(tempogram1)))
    
    
    
    #---------------------------------------------------------------------------------------------------------------------
    audio_waveform_split_angle = getArrayOfAnglesBetweenConsecutivepointsInArray(audio_waveform_split)
    chroma_stft_angle = getArrayOfAnglesBetweenConsecutivepointsInArray(chroma_stft)
    spectral_centroid_angle = getArrayOfAnglesBetweenConsecutivepointsInArray(spectral_centroid)
    spectral_bandwidth_angle = getArrayOfAnglesBetweenConsecutivepointsInArray(spectral_bandwidth)
            
    rolloff_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(rolloff) 
    zero_crossing_rate_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(zero_crossing_rate) 
    mfcc_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(mfcc) 
    rmse_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(rmse) 
    chroma_cqt_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(chroma_cqt) 
    chroma_cens_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(chroma_cens) 
    melspectrogram_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(melspectrogram) 
    spectral_contrast_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(spectral_contrast) 
    spectral_flatness_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(spectral_flatness) 
    poly_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(poly) 
    tonal_centroid_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(tonal_centroid) 
    tempogram_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(tempogram) 
    onset_detect_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(onset_detect) 
    onset_strength_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(onset_strength) 
    onset_strength_multi_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(onset_strength_multi) 
    beat_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(beat) 
    tempo_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(tempo) 
    harmonic_elements_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(harmonic_elements) 
    percussive_elements_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(percussive_elements) 
#     delta_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(delta) 
#     vocals_angle= getArrayOfAnglesBetweenConsecutivepointsInArray(vocals)
    
    #-------------------------------------------------------24--------------------------------------------------------------
    featurelist.append(pickle.dumps(audio_waveform_split,protocol = -1))
    featurelist.append(pickle.dumps(chroma_stft,protocol = -1))
    featurelist.append(pickle.dumps(spectral_centroid,protocol = -1))
    featurelist.append(pickle.dumps(spectral_bandwidth,protocol = -1))
    featurelist.append(pickle.dumps(rolloff,protocol = -1))
    featurelist.append(pickle.dumps(zero_crossing_rate,protocol = -1))
    featurelist.append(pickle.dumps(mfcc,protocol = -1))
    featurelist.append(pickle.dumps(rmse,protocol = -1))
    featurelist.append(pickle.dumps(chroma_cqt,protocol = -1))
    featurelist.append(pickle.dumps(chroma_cens,protocol = -1))
    featurelist.append(pickle.dumps(melspectrogram,protocol = -1))
    featurelist.append(pickle.dumps(spectral_contrast,protocol = -1))
    featurelist.append(pickle.dumps(spectral_flatness,protocol = -1))
    featurelist.append(pickle.dumps(poly,protocol = -1))
    featurelist.append(pickle.dumps(tonal_centroid,protocol = -1))
    featurelist.append(pickle.dumps(tempogram,protocol = -1))
    featurelist.append(pickle.dumps(onset_detect,protocol = -1))
    featurelist.append(pickle.dumps(onset_strength,protocol = -1))
    featurelist.append(pickle.dumps(onset_strength_multi,protocol = -1))
    featurelist.append(pickle.dumps(beat,protocol = -1))
    featurelist.append(pickle.dumps(tempo,protocol = -1))
    featurelist.append(pickle.dumps(harmonic_elements,protocol = -1))
    featurelist.append(pickle.dumps(percussive_elements,protocol = -1))
    featurelist.append(pickle.dumps(delta,protocol = -1))
    #----------------------------------------------------24-----------------------------------------------------------------
    featurelist.append(pickle.dumps(audio_waveform_split_angle,protocol = -1))
    featurelist.append(pickle.dumps(chroma_stft_angle,protocol = -1))
    featurelist.append(pickle.dumps(spectral_centroid_angle,protocol = -1))
    featurelist.append(pickle.dumps(spectral_bandwidth_angle,protocol = -1))
    featurelist.append(pickle.dumps(rolloff_angle,protocol = -1))
    featurelist.append(pickle.dumps(zero_crossing_rate_angle,protocol = -1))
    featurelist.append(pickle.dumps(mfcc_angle,protocol = -1))
    featurelist.append(pickle.dumps(rmse_angle,protocol = -1))
    featurelist.append(pickle.dumps(chroma_cqt_angle,protocol = -1))
    featurelist.append(pickle.dumps(chroma_cens_angle,protocol = -1))
    featurelist.append(pickle.dumps(melspectrogram_angle,protocol = -1))
    featurelist.append(pickle.dumps(spectral_contrast_angle,protocol = -1))
    featurelist.append(pickle.dumps(spectral_flatness_angle,protocol = -1))
    featurelist.append(pickle.dumps(poly_angle,protocol = -1))
    featurelist.append(pickle.dumps(tonal_centroid_angle,protocol = -1))
    featurelist.append(pickle.dumps(tempogram_angle,protocol = -1))
    featurelist.append(pickle.dumps(onset_detect_angle,protocol = -1))
    featurelist.append(pickle.dumps(onset_strength_angle,protocol = -1))
    featurelist.append(pickle.dumps(onset_strength_multi_angle,protocol = -1))
    featurelist.append(pickle.dumps(beat_angle,protocol = -1))
    featurelist.append(pickle.dumps(tempo_angle,protocol = -1))
    featurelist.append(pickle.dumps(harmonic_elements_angle,protocol = -1))
    featurelist.append(pickle.dumps(percussive_elements_angle,protocol = -1))
    featurelist.append(pickle.dumps(delta_angle,protocol = -1))
#     featurelist.append(pickle.dumps(vocals_angle,protocol = -1))
    



        
        ###-------Storing the feature-list  to  Pandas DataFarme--------------------
    df.loc[sr_no] = featurelist
    
        
    


0


C:\ProgramData\Anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [44]:
df.head()

,filename,artist,duration,albumartist,bitrate,comment,filesize,genre,title,track,...,tonal_centroid_angle,tempogram_angle,onset_detect_angle,onset_strength_angle,onset_strength_multi_angle,beat_angle,tempo_angle,harmonic_elements_angle,percussive_elements_angle,delta_angle
1,Bebe Rexha - Drinking About You (The Chainsmok...,Bebe Rexha,217.034914,The Chainsmokers Bebe Rexha,320.0,,8712257,Electronic,Drinking About You (The Chainsmokers Remix),None,...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...","b""\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
2,Bulleya - DJ LIJO & VIKAS J REMIX.mp3,,240.068006,None,320.0,,9608031,None,Bulleya - DJ LIJO & VIKAS J REMIX,None,...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...","b""\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...","b""\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
3,Chaiyya Chaiyya - DJ LIJO's REMIX.mp3,,248.139043,None,320.0,,9948637,None,Chaiyya Chaiyya - DJ LIJO's REMIX,None,...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
4,Coca Cola - Songs.pk - 128Kbps.mp3,"Tony Kakkar, Neha Kakkar, Young Desi",179.633623,None,128.0,None,3069797,Bollywood Music,Coca Cola - Songs.pk,2,...,"b""\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
5,Coca Cola - Songs.pk - 320Kbps.mp3,"Tony Kakkar, Neha Kakkar, Young Desi",179.535321,None,320.0,None,7377494,Bollywood Music,Coca Cola - Songs.pk,2,...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'


In [45]:
sr_no

6

In [47]:
df.to_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/split1_feature_extraction_mar_28_1.pkl",protocol = -1)
# df_new = pd.read_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/split1_feature_extraction_7.pkl")